In [9]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

__author__ = 'xiaosong Liu'

import os
import time
import uuid
import asyncio
import aiohttp
import aiofiles

semaphore = asyncio.Semaphore(5)

In [10]:
async def downloads_pic():
    url = 'https://passport.lagou.com/vcode/create?from=register&refresh=1451121012510'
    dirpath = 'pic_temp'
    filename = 'lagou-'
    filename = ''.join(filename,uuid.uuid1())
    pic_save_path = os.path.join(dirpath,filename)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36",
        "Referer": 'https://passport.lagou.com/'
    }

    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

    async with semaphore:
        async with aiohttp.ClientSession() as session:
            async with session.get(url,headers) as html:
                print(html.status)
                try:
                    pic_images = await html.read()
                except:
                    print(html.status)
                if pic_images:
                    f = await aiofiles.open(pic_save_path + '.bmp','wb')
                    await f.write(pic_images)
                   
                    return True
                else:
                    return False

In [14]:
start_time = time.time()

loop = asyncio.get_event_loop()
download_tasks = [downloads_pic(),downloads_pic()]
loop.run_until_complete(asyncio.wait(download_tasks))
 #解决 Event loop is closed 的报错
loop.close()

print('下载完成: 费时%s秒!' % str(end_time - start_time))


RuntimeError: Event loop is closed

In [20]:
print(uuid.uuid1())

7eb8617e-f8e2-11e7-8e86-5800e3c3e0ee


In [55]:
print(uuid.uuid1())

7e486a0c-f8ed-11e7-a78f-5800e3c3e0ee


RuntimeError: Event loop is closed

In [ ]:
import numpy as np
import cv2
import random

if __name__ == '__main__':

    img = cv2.imread("pic_temp\lagou-de840964-f8f3-11e7-967e-5800e3c3e0ee.bmp",4)
    h, w = img.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    seed_pt = None
    fixed_range = True
    connectivity = 4

    def update(dummy=None):
        if seed_pt is None:
            cv2.imshow('floodfill', img)
            return
        flooded = img.copy()
        mask[:] = 1
        lo = cv2.getTrackbarPos('lo', 'floodfill')
        hi = cv2.getTrackbarPos('hi', 'floodfill')
        flags = connectivity
        if fixed_range:
            flags |= cv2.FLOODFILL_FIXED_RANGE 

        cv2.floodFill(flooded, mask, seed_pt, (random.randint(0,255), random.randint(0,255), random.randint(0,255)), (lo,)*3, (hi,)*3, flags)

        cv2.circle(flooded, seed_pt, 2, (0, 0, 255), -1)#选定基准点用红色圆点标出
        cv2.imshow('floodfill', flooded)

    def onmouse(event, x, y, flags, param):#鼠标响应函数
        global seed_pt
        if flags & cv2.EVENT_FLAG_LBUTTON:#鼠标左键响应，选择漫水填充基准点
            seed_pt = x, y
            update()

    update()
    cv2.setMouseCallback('floodfill', onmouse)
    cv2.createTrackbar('lo', 'floodfill', 20, 255, update)
    cv2.createTrackbar('hi', 'floodfill', 20, 255, update)

    while True:
        ch = 0xFF & cv2.waitKey()
        if ch == 27:
            break
        if ch == ord('f'):
            fixed_range = not fixed_range #选定时flags的高位比特位0，也就是邻域的选定为当前像素与相邻像素的的差，这样的效果就是联通区域会很大
            print ('using %s range' % ('floating', 'fixed')[fixed_range])
            update()
        if ch == ord('c'):
            connectivity = 12-connectivity #选择4方向或则8方向种子扩散
            print('connectivity =', connectivity)
            update()
    cv2.destroyAllWindows()   